### Imports

In [ ]:
import os
import time

import numpy as np
import pandas as pd
from pathlib import Path

import string
import itertools

import sys
sys.path.append("..")

from simulation.simulation_utils import simulate
from simulation.simulation_tools import get_optimal_sim_XY

rng = np.random.default_rng()

COL_NAMES = list(string.ascii_uppercase) + ["".join(a) for a in list(itertools.permutations(list(string.ascii_uppercase), r=2))]

### Simple Simulation

In [7]:
# data
data_path = list(Path(".").resolve().parents)[1] / 'data' / 'cp_style' / 'increasing_edges_cp_1' / 'data'
fn = os.listdir(data_path)[0]
true_data = pd.read_csv(data_path / fn)

# TCS configuration
cfg = {
    "cd_method" : "PCMCI", 
    "cd_kwargs" : None, 
    "pred_method" : "TCDF", 
    "pred_kwargs" : None, 
    "o_approximation" : "spline", 
    "noise_approximation" : "spline",
}

# simulate
start_time = time.time()
sim_data, sim_scm, funcs_and_noise, scores = simulate(
    true_data=true_data, 
    true_label=None, 
    n_samples=500, 
    verbose=True, 
    **cfg
)
elapsed_time = time.time() - start_time
print(f"LOG : Simple Simulation : Elapsed time for single simulation: {round(elapsed_time, 2)}")

LOG : Causal structure : estimate_with_PCMCI w/ {'n_lags': 1, 'n_reps': 10} ...
LOG : Causal structure : estimate_with_PCMCI was successfully used.
LOG : Forecasting : Node A_t (has parents) ...


100%|██████████| 1000/1000 [00:00<00:00, 1258.22it/s]


LOG : Forecasting : Node B_t (has parents) ...


100%|██████████| 1000/1000 [00:00<00:00, 1224.80it/s]


LOG : Forecasting : Node C_t (has parents) ...


...

LOG : Forecasting : Node E_t (has parents) ...


100%|██████████| 1000/1000 [00:00<00:00, 1399.25it/s]


LOG : Forecasting : Node F_t (orphan) ...
LOG : Forecasting : Node G_t (orphan) ...
LOG : Forecasting : Node H_t (orphan) ...
LOG : Forecasting : Node I_t (orphan) ...
LOG : Forecasting : Node J_t (orphan) ...


100%|██████████| 520/520 [00:09<00:00, 53.75it/s]

LOG : Simple Simulation : Elapsed time for single simulation: 51.99


### Optimized Simulation

In [ ]:
# data
data_path = list(Path(".").resolve().parents)[1] / 'data' / 'fMRI' / 'timeseries'
fn = os.listdir(data_path)[0]
true_data = pd.read_csv(data_path / fn)

# simulate
start_time = time.time()
res = get_optimal_sim_XY(
        true_data = true_data, 
        CONFIGS = None, 
        done_eval = False,
        optimal_det_config = None,
        optimal_det_func = None, 
        sparsity_penalty=True,
        verbose = True
)
elapsed_time = time.time() - start_time
print(f"LOG : Simple Simulation : Elapsed time for optimized simulation: {round(elapsed_time, 2)}")

LOG: Optimal simulation: 8 are to be tested ...
_____________________________________________________________________________________________________________
_______________________ {'cd': {'cd_method': 'CP', 'cd_kwargs': {'model': '../cd_methods/CausalPretraining/res/deep_CI_RH_12_3_merged_290k.ckpt', 'model_name': 'deep_CI_RH_12_3_merged_290k', 'MAX_VAR': 12, 'thresholded': True, 'threshold': 0.05, 'enforce_density': False, 'density': [2, 10]}}, 'fc': {'pred_method': 'TCDF', 'pred_kwargs': {}}, 'z': 'spline', 'o': 'spline'} _______________________


100%|██████████| 24/24 [00:32<00:00,  1.37s/it]


LOG: Optimal Detection Config: svm_detection_XY: 0.9351095652173913 (auc) || configs: 24 || elapsed_time: 32.88 (s)


100%|██████████| 16/16 [02:48<00:00, 10.56s/it]


LOG: Optimal Detection Config: lstm_detection_XY: 0.6192131734068448 (auc) || configs: 16 || elapsed_time: 168.89 (s)
LOG: Optimal Simulation: Max C2ST AUC achieved: 0.9351067826086956
_____________________________________________________________________________________________________________
_____________________________________________________________________________________________________________
_______________________ {'cd': {'cd_method': 'CP', 'cd_kwargs': {'model': '../cd_methods/CausalPretraining/res/deep_CI_RH_12_3_merged_290k.ckpt', 'model_name': 'deep_CI_RH_12_3_merged_290k', 'MAX_VAR': 12, 'thresholded': True, 'threshold': 0.05, 'enforce_density': False, 'density': [2, 10]}}, 'fc': {'pred_method': 'TCDF', 'pred_kwargs': {}}, 'z': 'est', 'o': 'est'} _______________________


...

100%|██████████| 24/24 [00:27<00:00,  1.15s/it]


LOG: Optimal Detection Config: svm_detection_XY: 0.955330255247747 (auc) || configs: 24 || elapsed_time: 27.54 (s)


100%|██████████| 16/16 [02:57<00:00, 11.08s/it]


LOG: Optimal Detection Config: lstm_detection_XY: 0.5193907563025211 (auc) || configs: 16 || elapsed_time: 177.21 (s)
LOG: Optimal Simulation: Max C2ST AUC achieved: 0.9553260881643719
_____________________________________________________________________________________________________________
_____________________________________________________________________________________________________________
_______________________ {'cd': {'cd_method': 'PCMCI', 'cd_kwargs': {'n_lags': 3, 'n_reps': 10}}, 'fc': {'pred_method': 'TCDF', 'pred_kwargs': {}}, 'z': 'spline', 'o': 'spline'} _______________________


 12%|█▎        | 3/24 [41:00<6:33:02, 1122.96s/it]